# Particle-flow interaction


We split the particle and flow solvers into separate ranks similar to the approach taken in Thari et al's [ATEL paper](./Resources.ipynb#atel):

![Solver structure](./images/solver-structure.png)
<img src="./images/solver-structure.png">

## Additional source terms for the flow solver, provided by the particle solver
The Lagrangian (particle) simulation influences the flow solver by providing addition source terms for the differential equations in the flow solver.
These source terms are:
* $\frac{dm_d}{dt}$ ($\dot{m_d}$), which is added as a source term in the Mass Continuity Equation
* $\frac{d \omega_Z}{dt}$ ($\dot{\omega_Z}$), which is added as a source term in the Mixture Fractioon equation
* $S_{i}$, added to the Momentum Equation
* $Q_{d}$, added to the Energy Equation

Formulas for these are given in Thari's paper.


## Flow field values for the particle solver, provided by the flow solver
TODO!

# Synchronisation and timestepping
In the ATEL approach, we need to synchronise flow field values
In other words, the particle solver is using flow field values in the past. This does not drastically affect the outcome of the simulation, but
does allow us to parallelise the problem better.

In the ATEL approach, the synchronisation uses one-sided MPI comminicaction between shared memory regions on a single compute node, so that solver rank A
can read values written by the solver on rank B, without communication bottlenecks. In MiniCombust, we use a slightly different approach and
duplicate the flow field values in the particle solver, and the source term fields in the flow solver, i.e. we use a true distributed memory approach.
This allows us the flexibility to have particle ranks that gather flow values from other, possibly distant ranks, and allows much more flexible implementation
of different load balancing strategies.

In [1]:

from collections import namedtuple
from typing import Map

"""
NOTE: The spray solver decomposition is not necessarily the same as the particle solver decomposition!
So when we update the spray field, we need to know which cell's spray is calculated by which rank
And we need to know which cell's flow is calculated by which rank
"""
class SpraySolver:
    flow_values = {
        'turbulence_field': None,
        'combustion_field': None,
        'spray_field': None,
        'flow_field': None,
    }

    def update_spray_field(self):
        """
        Copies in the new grid source terms from the particle solver
        We COPY rather than use Read-only access to a Window as in ATEL, to allow
        the flexibility of different load balancing strategies
        """
        pass

    def solve_combustion_equations(self):
        """
        transport of mixture fraction
        progress variable
        variance of mixture fraction
        variance of progress variable
        """
        pass

    def update_combustion_field(self):
        """
        interpolate thermomech state of cell from 
        mixture fraction and progress variable, looking up
        against FGM tables
        """
        pass

    def solve_turbulence_equations(self):
        """
        turbulent kinetic energy
        dissipation
        enthalpy?
        """
        pass

    def update_turbulence_field(self):
        """
        
        """
        pass

    def solve_flow_equations(self):
        """
        conservation of mass
        conservation of momentum
        conservation of energy
        pressure correction
        """
        pass

    def timestep(self):
        self.update_spray_field() # syncrhonising with particle solver
        self.solve_combustion_equations()
        self.update_combustion_field()
        self.solve_turbulence_equations()
        self.update_turbulence_field()
        self.solve_flow_equations()
        self.update_flow_field()



class ParticleSolver:
    Particle = namedtuple('Particle', ['coords', 'mass', 'diameter', 'rate_of_mass_change', ''])

    spray_values = {
        'flow_field': None,
        'particles': Map[int, Particle]
    }

    derived_additional_source_terms = {
        'dm_d_dt' : None, # for mass equation
        'domega_Z_dt': None, # for mixture fraction equation
        'S_i,d': None, # for momentum equation
        'Q_d': None # for energy equation
    }

    def update_flow_field(self):
        """
        Copies in the velocity for cells from the flow solver
        This will be used in the calculation of the 'virtual force' in solve_spray_equations
        """
        pass

    def particle_release(self):
        """
        At spray injection locations, new droplets are injected with diameter from
        Rosin-Rammler distribution, and corresponding properties: velocity, 
        """
        pass

    def solve_spray_equations(self):
        """
        evaporation: update mass
        drag force
        virtual force
        body force 
        solve velocity
        """

        pass

    def update_particle_positions(self):
        """
        Move particle according to calculated velocity and current position
        """
        particles.positions += particles.velocities * timestep_size

        pass

    def update_spray_source_terms(self):
        """
        For each cell in domain we calculate
            dm_d_dt = sum_over_all_particles(dm_d/d_t for particle)
            domega_Z_dt = dm_d_dt
            `S_i,d` =  sum_over_all_particles(d(droplet_mass * droplet_velocity)/dt)
            `Q_d` = sum_over_all_particles(heat_transferred_from_air_to_fuel - heat_absorbed_through_evaporation)
        """

        pass

    def map_source_terms_to_grid(self):
        """
        Copy source terms to grid cells ready for transfer to spray solver
        """
        pass

    def timestep(self):
        self.update_flow_field() # synchronising with flow solver
        self.particle_release()
        self.solve_spray_equations()
        self.update_particle_positions()
        self.update_spray_source_terms()
        self.map_source_terms_to_grid()

ImportError: cannot import name 'Map' from 'typing' (/Users/work/.pyenv/versions/3.7.9/lib/python3.7/typing.py)

# Interpolation between cells and particles for flow field values

When we read the field value (e.g. pressure) for a cell that a particle is in, we get inaccuraccies if we only use the cell centre value and don't compensate for the particle's position in the cell.
As a correction, we first interpolate the field to values at the cell nodes. Then, when calculating the value for a particle, we interpolate the nodal values of the cell to the particle's position.

## Interpolation of cell centre values to nodal values
We correct using the gradient of the field and the distance from the node to the cell centre, and normalising nodal values based on `Dolfyn: opendx.f90:: InterpolateData`
For non-scalar fields (velocity), this is done _per dimension_

In [ ]:

def interpolate_data(Φ: NDArray[T]) -> NDArray[T]:
    """
    Interpolates data from scalar field Φ at cell centres to values at the node coordinates of each cell
    NOTE We only do Dolfyn's 'Mode 0' interpolation - i.e. we DO use boundary face values

    phi: len num_cells + num_boundaries, 1D  SOME SCALAR FIELD

    Dolfyn: opendx.f90:: InterpolateData
    """
    dΦdX = gradient(Φ)
    nodal_values_of_Φ = np.zeros((num_nodes, ), dtype=T)
    nodal_counter = np.zeros((num_nodes, ), dtype=int)

    # Project each cells value to the faces' nodes by using the gradient and distance from the node
    for cell in Cells:
        cell_coordinates = cell.coordinates()
        cell_φ = Φ[cell.id()]
        cell_dφ = dΦdX[cell.id()]
        for face in cell.faces():
            for node in face.nodes():
                ds = node.coordinates() - cell_coordinates
                nodal_values_of_Φ[node.id()] += cell_φ + np.dot(cell_dφ, ds)
                nodal_counter[node.id()] += 1

    # now we take into account boundaries
    for boundary in Boundaries:
        if boundary.region().type() != SYMMETRIC_PLANE:
            for node in enumerate(boundary.face().nodes()):
                nodal_values_of_Φ[node.id()] += Φ[Cells.num_cells + boundary.id()]
                nodal_counter[node.id()] += 1
        
    # Normalise nodal values by how many times node was incremented
    for node in Nodes:
        if nodal_counter[node.id()] > 0:
            nodal_values_of_Φ[node.id()] /= nodal_counter[node.id()]
    
    return nodal_values_of_Φ

## Interpolating from nodal values to a corrected value of a field for the particle
Weight by the normalised inverse square distance of the particle to each node of the cell. `Dolfyn: particles.f90::ParticleGetVelocity`

In [ ]:
# This should work for 3D (e.g. velocity) and 1D (e.g. scalar) values
# because of numpy broadcasting rules
def get_corrected_value_for_particle_in_cell(position: T3, cell_node_ids: NDArray[int], values_at_all_nodes: NDArray[T], all_node_coords: NDArray[T]) -> T3:
    """
    Rather than using cell-centered scalar value (e.g temperature), we calculate the cell value for each particle by using the 
    values at the nodes of the cell (as determined by interpolate_data above), and weighting by the distance of the particle to each node

    Dolfyn: particles.f90::ParticleGetVelocity
    """
    epsilon = 1e-12 # a small value, to avoid division by 0 
    diff_particle_to_nodes = position - all_node_coords[cell_nodes, :] # Delta from particle position to each node's position
    node_weights = np.dot(diff_particle_to_nodes, diff_particle_to_nodes) # square distance
    node_weights = 1. / (node_weights + epsilon) # inverse
    total = sum(node_weights) # total weight

    node_weights /= total # normalise weights
    return np.sum(node_weights * values_at_all_nodes[cell_node_ids,:], ax=0) # sum weighted node velocities


# Interpolating from Particle values in a cell, to a cell-centred source term
In each cace, we simply sum the corresponding values for each particle in the cell

In [2]:
def particle_rate_of_mass(particle):
    # Todo Do we calculate this term as part of particle_intercace?
    pass

def momentum(particle):
    # Todo do we calculate this term as part of particle_interface?
    """ d/dt of particle.mass * particle.velocity"""
    pass

def energy(particle):
    return particle.heat_transferred_from_air_to_fuel - particle.heat_absorbed_through_evaporation


# ...

for cell in Cells:
    particles_in_cell = filter(lambda p: p.current_cell() == cell, Particles)
    rate_of_mass_source_term[cell.id()] = sum(particle_rate_of_mass(particle) for particle in particles_in_cell)
    momentum_source_term[cell.id()] = sum(momentum(particle) for particle in particles_in_cell)
    energy_source_term[cell.id()] = sum(energy(particle) for particle in particles_in_cell)
mixture_fraction_source_term[:] = rate_of_mass_source_term[:]
# ...



NameError: name 'Cells' is not defined